In [ ]:
from typing import List

from datasets import load_dataset

from fastrepl.run import completion
from fastrepl.eval.metric.metric import BaseMetric, HuggingfaceMetric
from fastrepl.polish.updatable import Updatable
from fastrepl.loop import ClassificationRunner
from fastrepl.helpers import Experiment, setenv

In [ ]:
def classify(input: str):
    system = Updatable(
        "SYSTEM", "Output only 'POSITIVE' or 'NEGATIVE' for the user's input."
    )

    res = completion(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": str(system)},
            {"role": "user", "content": input},
        ],
        temperature=0,
    )
    return res["choices"][0]["message"]["content"]

In [ ]:
def prepare_dataset():
    imdb = load_dataset("imdb", split="train")
    imdb = imdb.shuffle(seed=13).select([i for i in list(range(10))])
    return imdb

In [ ]:
setenv("fastrepl", True)

with Experiment(GRAPH=2) as manager:
    ds = prepare_dataset()
    ds = [{"input": [d["text"]], "output": d["label"]} for d in ds]

    metrics: List[BaseMetric] = [
        HuggingfaceMetric("accuracy"),
        HuggingfaceMetric("precision"),
        HuggingfaceMetric("matthews_correlation"),
    ]

    runner = ClassificationRunner(
        fn=classify,
        label_mapping={"NEGATIVE": 0, "POSITIVE": 1},
    )

    report = runner.eval(dataset=ds, metrics=metrics)
    print(report)

    report = runner.fit(dataset=ds, metrics=metrics)
    print(report)